In [2]:
import os
import folium
import json
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.externals import joblib
min_max_scaler = preprocessing.MinMaxScaler()

In [3]:
#import geojson file with location data
china = os.path.join('china_provinces_en.json')
geo_json_data = json.load(open(china))

In [4]:
#Read data for city and their agricultural production & restaurant amount
Geo = pd.read_csv('LocationGeo.csv')

In [5]:
#Convert agricultural production to approximate biowaste amount based on hainan data
Geo['input'] = Geo['2016']*10000/334
Geo['input'] = Geo['input'].round(2)

In [6]:
#colormap for agricultural production
from branca.colormap import linear
colormap = linear.YlGn.scale(Geo['2016'].min(),Geo['2016'].max())#color:Y1Gn
print(colormap(5.0))
colormap

#ffffcc


In [7]:
#colormap for restaurant amount
colormap1 = linear.PuBu.scale(Geo['res_2016'].min(),Geo['res_2016'].max())
print(colormap1(5.0))
colormap1

#f1eef6


In [8]:
#import longitude and latitude data for capital of province
locations = Geo[['longitude','latitude']]
locationlist = locations.values.tolist()

In [9]:
#agricultural data for each province
ag_dict = Geo.set_index('city')['2016']

In [10]:
#restaurant data for each province
res_dict = Geo.set_index('city')['res_2016']

In [11]:
#Estimate waste amount based on percentage obatined from Hainan's data
Waste_perc = {'Pig_Manure_t':0.082090753,'Cassava_t':0.052062101,'Fish_waste_water_t':0.011827662,'Kitchen_food_waste_t':0.090209462,'Municipal_fecal_residue_t':0.15976846,'Tea_waste_t':0.005256164,'Chicken_litter_t':0.038159617,'Bagasse_feed_t':0.071323661,'Alcohol_waste_t':0.003800583,'Chinese_medicine_waste_t':0.001210285,'Energy_grass_t':0.000385748,'Banana_fruit_shafts_t':0.002775097,'Lemon_waste_t':0.002088569,'Percolate_t':0.463951801,'Other_waste_t':0.015090037}

In [12]:
#Calculate input for each kind of waste
for i in Waste_perc:
    Geo[i]=Geo['input'].apply(lambda x: x*Waste_perc[i])

In [13]:
#Extract different input values
local = Geo[['Pig_Manure_t','Cassava_t','Fish_waste_water_t','Kitchen_food_waste_t','Municipal_fecal_residue_t','Tea_waste_t','Chicken_litter_t','Bagasse_feed_t','Alcohol_waste_t','Chinese_medicine_waste_t','Energy_grass_t','Banana_fruit_shafts_t','Lemon_waste_t','Percolate_t','Other_waste_t']]

In [14]:
#Add 1/x feature
for col in local.columns:
    local['1/'+col] = 1/(local[col])
local.replace(float('inf'), 0, inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
#add x^2 feature
for col in local.columns:
    local[col+'**2'] = (local[col])**2

In [16]:
#load Hainan model
knn = joblib.load('hainan_model.pkl')

In [17]:
#Normalize the features
np_scaled_local = min_max_scaler.fit_transform(local)

In [18]:
local_normalized = pd.DataFrame(np_scaled_local)

In [19]:
#Generate Prediction
Y = knn.predict(local_normalized)

In [20]:
#Scale up production
Biogas = Y*15100*365/5
Biogas = Biogas.flatten().round(2)

In [21]:
#Change to string type so that can show on map
Biogas_str = Biogas.astype(str)

In [22]:
#build map
m = folium.Map([41,100], zoom_start = 4)
for point in range(0,len(locationlist)):
    folium.Marker(location = locationlist[point], popup = str('The potential Bio Gas Production is  '+ Biogas_str[point]+' dollars per year')).add_to(m)
folium.GeoJson(geo_json_data, name = 'agricultural_production', 
               style_function = lambda feature:{'fillColor':colormap(ag_dict[feature['id']]),
                                                'color':'black','weight':1,'dashArray':'5,5','fillOpacity':0.5}).add_to(m)
folium.GeoJson(geo_json_data, name = 'restaurant_amount',
             style_function = lambda feature:{'fillColor':colormap1(res_dict[feature['id']]),
                                             'color':'purple','weighth':1,'dashArray':'5.5','fillOpacity':0.5}).add_to(m)
folium.LayerControl().add_to(m)
m.save(os.path.join('GeoJSON_and_choropleth_5.html'))
m